# Evaluate Semantic Kernel OpenAI Responses Agents in Azure AI Foundry

## Objective

This sample demonstrates how to evaluate Semantic Kernel OpenAIResponses agent in Azure AI Foundry. It provides a step-by-step guide to set up the environment, create an agent, and evaluate its performance.

## Time
You can expect to complete this sample in approximately 20 minutes.

## Prerequisites
### Packages
- `semantic-kernel` installed (`pip install semantic-kernel`)
- `azure-ai-evaluation` SDK installed
- An Azure OpenAI resource with a deployment configured

### Environment Variables
- For AzureChatService:
  - `AZURE_OPENAI_API_KEY`
  - `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
  - `AZURE_OPENAI_ENDPOINT`
  - `AZURE_OPENAI_API_VERSION="2025-04-01-preview"`
- For evaluating agents:
  - `PROJECT_CONNECTION_STRING`
  - `AZURE_OPENAI_ENDPOINT`
  - `AZURE_OPENAI_API_KEY`
  - `AZURE_OPENAI_API_VERSION`
  - `MODEL_DEPLOYMENT_NAME`
- For Azure AI Foundry (Bonus):
  - `AZURE_SUBSCRIPTION_ID`
  - `PROJECT_NAME`
  - `RESOURCE_GROUP_NAME`

### Create an OpenAIResponsesAgent with a plugin - [reference](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-types/responses-agent?pivots=programming-language-python)

In [1]:
from typing import Annotated

from semantic_kernel.agents import AzureAssistantAgent
from semantic_kernel.connectors.ai.open_ai import AzureOpenAISettings
from semantic_kernel.functions import kernel_function


# Define a sample plugin for the sample
class MenuPlugin:
    """A sample Menu Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of specials from the menu.")
    def get_specials(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    @kernel_function(description="Provides the price of the requested menu item.")
    def get_item_price(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns the price of the menu item."]:
        return "$9.99"


# Create an agent
client = AzureAssistantAgent.create_client()
definition = await client.beta.assistants.create(
    model=AzureOpenAISettings().chat_deployment_name,
    instructions="Answer questions about the menu.",
    name="Assistant",
)
agent = AzureAssistantAgent(
    client=client,
    definition=definition,
    plugins=[MenuPlugin()],
)

### Invoke the agent

In [2]:
USER_INPUTS = [
    "Hello",
    "What is the special soup?",
    "What is the special drink?",
    "How much is it?",
    "Thank you",
]

thread = None
for user_input in USER_INPUTS:
    print(f"## User: {user_input}")
    response = await agent.get_response(messages=user_input, thread=thread)
    print(f"## {response.name}: {response.content}")
    thread = response.thread

## User: Hello
## Assistant: Hello! How can I assist you today?
## User: What is the special soup?
## Assistant: The special soup is Clam Chowder. Would you like to know more about it or any other special items?
## User: What is the special drink?
## Assistant: The special drink is Chai Tea. Would you like more details or information about anything else on the menu?
## User: How much is it?
## Assistant: The price of the Chai Tea is $9.99. Is there anything else you would like to know?
## User: Thank you
## Assistant: You're welcome! If you have any more questions in the future, feel free to ask. Have a great day!


### Converter: Get data from agent

In [6]:
import os

from azure.ai.evaluation import AIAgentConverter
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    endpoint="https://sc-ly6269080-1392-resource.services.ai.azure.com/api/projects/sc-ly6269080-1392",
    credential=DefaultAzureCredential(),
)
converter = AIAgentConverter(project_client)

file_name = "evaluation_data.jsonl"
# Save the agent thread data to a JSONL file (all turns)
evaluation_data = await converter.prepare_evaluation_data([thread.id], filename=file_name)
# print(json.dumps(evaluation_data, indent=4))
len(evaluation_data)  # number of turns in the thread

ResourceNotFoundError: (None) No thread found with id 'thread_PEshQVS2Kqv0t0uRRzmLvESb'.
Code: None
Message: No thread found with id 'thread_PEshQVS2Kqv0t0uRRzmLvESb'.

### Setting up evaluator

We will select the following evaluators to assess the different aspects relevant for agent quality: 

- [Intent resolution](https://aka.ms/intentresolution-sample): measures the extent of which an agent identifies the correct intent from a user query. Scale: integer 1-5. Higher is better.
- [Tool call accuracy](https://aka.ms/toolcallaccuracy-sample): evaluates the agent’s ability to select the appropriate tools, and process correct parameters from previous steps. Scale: float 0-1. Higher is better.
- [Task adherence](https://aka.ms/taskadherence-sample): measures the extent of which an agent’s final response adheres to the task based on its system message and a user query. Scale: integer 1-5. Higher is better.


In [ ]:
import os
from pprint import pprint

from azure.ai.evaluation import (
    AzureOpenAIModelConfiguration,
    IntentResolutionEvaluator,
    TaskAdherenceEvaluator,
    ToolCallAccuracyEvaluator,
)

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)

intent_resolution = IntentResolutionEvaluator(model_config=model_config)

tool_call_accuracy = ToolCallAccuracyEvaluator(model_config=model_config)

task_adherence = TaskAdherenceEvaluator(model_config=model_config)

Class IntentResolutionEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Class ToolCallAccuracyEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class TaskAdherenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### Run Evaluator

In [ ]:
from azure.ai.evaluation import evaluate

response = evaluate(
    data=file_name,
    evaluators={
        "tool_call_accuracy": tool_call_accuracy,
        "intent_resolution": intent_resolution,
        "task_adherence": task_adherence,
    },
    azure_ai_project={
        "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
        "project_name": os.environ["PROJECT_NAME"],
        "resource_group_name": os.environ["RESOURCE_GROUP_NAME"],
    },
)
pprint(f"AI Foundary URL: {response.get('studio_url')}")

## Inspect results on Azure AI Foundry

Go to AI Foundry URL for rich Azure AI Foundry data visualization to inspect the evaluation scores and reasoning to quickly identify bugs and issues of your agent to fix and improve.

In [ ]:
# alternatively, you can use the following to get the evaluation results in memory

# average scores across all runs
pprint(response["metrics"])